In [2]:
# %pip install torch
# %pip install torchsummary

import torch
# from torchsummary import summary

print(f'available devices: {torch.cuda.device_count()}')
print(f'cuda is available: {torch.cuda.is_available()}')
!which python
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


available devices: 1
cuda is available: True
/home/kristina/miniconda3/bin/python


In [3]:
%pip install imutils

import os
import re
import cv2

import math
import  importlib # for refreshing import(nets.py)
import numpy as np
import matplotlib.pyplot as plt

# generate
from imutils import rotate_bound
from math import ceil
from random import randint, shuffle

# files
import utils 
import nets 
import dataset 

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Load the dataset

train_data_path= "../../data/imgs/wafer/in/"
wafer_img_paths=[]
for root, dirs, files in os.walk(train_data_path):
    for name in files:
        if re.findall('.jpg', name):
            wafer_img_paths.append(root+name)
    break
            
if not wafer_img_paths:
    print("err loading ",train_data_path)
else:
    print(len(wafer_img_paths))

101


In [68]:
image = cv2.imread(wafer_img_paths[0])
imageMask = utils.getMask(image,threshold=100,kernelSize=5) 
contours = cv2.findContours(imageMask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE) # img, contours, hierarchy

labelsCount, _, values, centroids=  cv2.connectedComponentsWithStats(imageMask) 
print(labelsCount, values.shape, centroids.shape)
contoursImg = np.zeros(image.shape, dtype=np.uint8)
trainFeatures= []

for i,cnt in enumerate(contours[0]): # iterate through contours
    if  i > 0:  # skip background (and small dust cv2.arcLength(contour, closed=True) < 50)
        contour = cnt.squeeze()
        
        features = extractFeaturesFromContour(contuour)
        contoursImg = cv2.drawContours(contoursImg, contours[0],i,(255,255,255), thickness=cv2.FILLED)

        print( features.shape)
        break

# plot_intensity_histogram(image) 

45 (45, 5) (45, 2)
(8,)


In [ ]:
image = cv2.imread(wafer_img_paths[0])
imageMask = utils.getMask(image,threshold=100,kernelSize=5) 
# listContours = markContours(imageMask) 
imgContours = utils.img2segmented(imageMask)

plt.imshow(imgContours)
plt.show()
a=np.unique(imgContours)
print(len(a))
print(imgContours.shape)
# TODO check 32.jpg

In [ ]:
importlib.reload(nets)
importlib.reload(utils)

batch_size=2
ds=[]
targets=[]
weights=[]
for idx, img in enumerate(wafer_img_paths):
    if idx >= batch_size:
        break
        
    image = cv2.imread(img)
    image = cv2.resize(image, dsize=( 1920,1088), interpolation=cv2.INTER_CUBIC)
    imgContoursBinary = utils.getMask(image,threshold=100,kernelSize=5)
    imgContoursSlices = np.array(utils.markContours(imgContoursBinary))
#     imgContours = img2segmented(getMask(image,threshold=100,kernelSize=5) )
    weightImg= utils.distanceMap(imgContoursBinary)
    
    ds.append(image)
    targets.append(imgContoursBinary) # imgContours
    weights.append(weightImg)
dataS=np.array(ds)
targetS=np.array(targets)
weightS=np.array(targets)

print("model input shape:",dataS[0].shape)
print("model GT shape:",targetS[0].shape)

In [ ]:
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler

importlib.reload(nets)
importlib.reload(dataset)

# activation_fn = nn.Softmax
batch_size = 2
epochs = 2

model = nets.UNet(
    enc_chs=(3,8,16),
    dec_chs=(16,8),
    num_class=1,
    retain_dim=True,
    out_sz=(1088, 1920)
).to(device)
optimizer = optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()# torch.nn.MSELoss()    #torch.nn.CrossEntropyLoss() #    
# loss = nn.CrossEntropyLoss(reduction='none')(output, target)
scaler = GradScaler()
# summary(model, (3, 1080 , 1920))

# TODO: the number of class labels was not specified correctly
Dataset = torch.tensor(dataS/255, dtype = torch.float, device = device).permute(0, 3, 1, 2)
Targets =  torch.tensor(targetS[:,None,:,:]/255, dtype = torch.float, device = device)
# Weights =  torch.tensor(weightS[:,None,:,:]/255, dtype = torch.float, device = device)
print(Targets.size())
running_loss = 0.0
for epoch in range(epochs):
    inputs = Dataset
    groundtruth = Targets

    optimizer.zero_grad()
    with autocast():
        outputs = model(inputs)
        loss = criterion( outputs, groundtruth)
#         loss = torch.mean(loss * Weights)
        
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    # print statistics
    running_loss += loss.item()
    print(f'[{epoch + 1}] loss: {running_loss / 100:.3f}')
    running_loss = 0.0


In [ ]:
from matplotlib.pyplot import figure

y= model(Dataset)
predictions = y.cpu().detach().numpy()

for i,im in enumerate(predictions):
    img=cv2.cvtColor(im.squeeze(),cv2.COLOR_GRAY2RGB)
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    orig= np.array(Dataset[i].permute( 1, 2, 0))
    orig = cv2.normalize(orig, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    gt = np.array(groundtruth[i]).squeeze()
    gt =cv2.cvtColor(gt,cv2.COLOR_GRAY2RGB)
    gt = cv2.normalize(gt, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    horiz = np.concatenate((orig,img,gt),axis=1)
    
    figure(figsize=(12, 6), dpi=80)
    plt.imshow(horiz)
    plt.show()

In [ ]:
# print(train_data_path)
# train_data_path =os.path.dirname(train_data_path)
# GROUND_MASK_DIR_PATH = train_data_path
# GROUND_MASK_DIR_PATH = os.path.join(os.path.dirname(train_data_path),"GT")


In [ ]:

# class arrDataset:
# #     normalize when loADING
#   def __init__(self, data_points_count= 2047):    
#     self.data_points_count= data_points_count
#     self.data = np.zeros((0, data_points_count))   
#     self.labels = np.zeros((0))
#     self.classes = {}
    
#   def add_items(self, data, labels):
#     data = data[:, 0:self.raman_points_count]
#     self.data = np.concatenate((self.data, data))
#     self.labels = np.concatenate((self.labels, labels))
    
#   def add_class(self, label, meas_class):
#     self.classes[label] =   meas_class

# def create_dataset(measurments,label, label_class):
#   dataset=Dataset()
#   measurments = [i for i in measurments if i is not None]
  
#   dataset.add_class(label, label_class)
  
#   for measurment in measurments:
#       dataset.add_items(np.array(measurment[None,:]),np.array([label]))
        
# #   print(dataset.get_data_shape())
#   return dataset

In [ ]:

#         if re.findall('.jpg$', imgPath)  :
#             path,ext  = os.path.splitext(imgPath)
#             file = os.path.basename(path)
#             dirPath = os.path.dirname(path)

#loss type                 input  target
#nn.MSELoss()              float, float
#nn.CrossEntropyLoss()     float, float 

In [ ]:
for parameter in model.parameters():
    print(parameter)